In [1]:
project_root = os.getcwd()
sys.path.append(project_root + '/modules')
# Pipeline의 구조를 이해하기 쉽게 하기 위해 작성한 ipynb 파일입니다
from Pipeline import pipe_eval
pipe = pipe_eval(verbose = True) # verbose를 False로 하면 print가 되지 않습니다.
# collection_name = 'chrdb.db',
# persist_directory = 'DB',
# mode = 'NaverCloudEmb'
# topk = 5
# verbose = True : answer과 score을 보여줍니다. 보고싶지 않다면 False로 하면 됩니다.
# default로 지정돼 있습니다. 클래스 선언 때 건네주면 바뀝니다.
pipe.setup(model = 'Exaone')
#                   가능한 모델 (default GPT)
#                   1. GPT
#                   2. Exaone
#                   3. Qwen14B
#                   4. Qwen32B

ModuleNotFoundError: No module named 'Pipeline'

In [3]:
query = 'SK하이닉스의 2025~2027년 주주환원 정책에서 변경된 고정배당금과 추가 배당금의 활용 방안은 무엇인가요?'
# validation데이터셋의 첫번째 question
retrieval_result = pipe.Q(query, mode = 'ensemble')
# bm25, dpr, ensemble 가능 (default 'ensemble')


page_content='SK하이닉스는 향후 3개년간 주주환원 정책을 공시하며, 연간 고정배당금을 1,500원으로 상향하고,  Free Cash Flow의 5% 재원을 재무 건전성 강화에 우선 활용한다고 밝혔다. 이는 메모리 업황 특성상 변동성이 큰 상황에서 내린 합리적인 결정이라고 판단된다.' metadata={'company_name': 'SK하이닉스', 'dir_route': 'datas/SK하이닉스_하나증권(2024.11.28)', 'file_name': 'SK하이닉스_하나증권(2024.11.28).pdf', 'id': 17, 'investment': '하나증권(2024.11.28)', 'original_content': 'SK하이닉스는 장 종료 후에 향후 3개년간 주주환원 쟁책을 공시했다. 기존 3개년 주주환 원 정책에서 변화가 생겼다. 변경된 부분의 핵심은 연간 고정배당금을 기존 1,200원에서 1,500원으로 상향하고, 기존 정책 하에서 지급하던 추가 배당금이었던 연간 Free Cash Flow의 5% 재원을 재무 건전성 강화에 우선 활용한다는 점이다. 아울러 정책 종료 후 총 재원 안에서 재무 건전성 목표가 달성되면 추가 환원을 실행하거나, 유의미한 수준의 Free Cash Flow 창출이 예상되는 경우에 조기 환원도 가능하다고 언급했다. 다시 설명하 자면 2025~2027년까지 3년 누적 Free Cash Flow의 50% 수준을 총 재원으로 설정하는 것은 기존과 동일하다. 다만, 기존에 매년 지급하던 FCF 내 5%의 추가 배당금을 앞으로 는 재무 건전성 강화로 사용하는 대신에 정책 종료 후 FCF 50% 수준 안에서 여유 재원을 기반으로 한 추가 환원을 결정한 것이다. 메모리 업황 특성상 변동성이 크기 때문에 예측 가능한 주주환원 규모를 선정하기 어렵기 때문에 3년 누적의 Free Cash Flow 선정은 동종 업체들의 정책과도 일맥상통하는 부분이다. 또한, SK하이닉스의 재무 건전

In [6]:
answer = pipe.A(query, retrieval_result)
# query 와 retrieval 결과를 받아 answer을 냅니다.



다음 참고 문서를 반드시 활용하여 질문에 대한 정확하고 간결한 답변을 작성하세요. 
답변은 반드시 질문과 관련된 사실에 기반하며, 불필요한 반복이나 모호함 없이 핵심 내용을 포함해야 합니다. 
100자를 초과하지 않도록 작성하세요.

질문: SK하이닉스의 2025~2027년 주주환원 정책에서 변경된 고정배당금과 추가 배당금의 활용 방안은 무엇인가요?
참고 문서1: SK하이닉스는 장 종료 후에 향후 3개년간 주주환원 쟁책을 공시했다. 기존 3개년 주주환 원 정책에서 변화가 생겼다. 변경된 부분의 핵심은 연간 고정배당금을 기존 1,200원에서 1,500원으로 상향하고, 기존 정책 하에서 지급하던 추가 배당금이었던 연간 Free Cash Flow의 5% 재원을 재무 건전성 강화에 우선 활용한다는 점이다. 아울러 정책 종료 후 총 재원 안에서 재무 건전성 목표가 달성되면 추가 환원을 실행하거나, 유의미한 수준의 Free Cash Flow 창출이 예상되는 경우에 조기 환원도 가능하다고 언급했다. 다시 설명하 자면 2025~2027년까지 3년 누적 Free Cash Flow의 50% 수준을 총 재원으로 설정하는 것은 기존과 동일하다. 다만, 기존에 매년 지급하던 FCF 내 5%의 추가 배당금을 앞으로 는 재무 건전성 강화로 사용하는 대신에 정책 종료 후 FCF 50% 수준 안에서 여유 재원을 기반으로 한 추가 환원을 결정한 것이다. 메모리 업황 특성상 변동성이 크기 때문에 예측 가능한 주주환원 규모를 선정하기 어렵기 때문에 3년 누적의 Free Cash Flow 선정은 동종 업체들의 정책과도 일맥상통하는 부분이다. 또한, SK하이닉스의 재무 건전성이 경쟁사들보 다 열위에 있던 점을 고려하면 금번 주주환원 정책 발표는 합리적인 결정이라 판단한다.
참고 문서2: 이번에 발표된 향후 3년간 주주환원 정책과 기업가치 제고 계획에 대해 다소 실망스러운 시장 반응이 있을 수 있다. 다만, 메모리 업종은 주주환원보다 기술 리더십이 우선시 되어 야 한다. 그럼에도 불구하고 변동성 높

In [7]:
answer

'2025~2027년 SK하이닉스 주주환원 정책은 연간 고정배당금 1,500원을 지급하고, 3년 누적 Free Cash Flow(Fcf)의 50%를 활용해 추가 환원을 진행할 계획입니다.'

In [4]:
answer = pipe.QA(query, mode = 'ensemble')
# QA를 따로 내는 것이 아니라 한번에 정답을 얻을 수도 있습니다. 이후 정량평가에서는 QA를 사용하면 됩니다.

In [2]:
generated_answer, G_retrieval_score, G_generation_score = pipe.QA_eval(mode = 'ensemble', sampling = True)

# validation 데이터셋의 무작위 샘플을 가져와 G-eval로 평가할 수 있습니다.

page_content='카카오뱅크는 대출 갈아타기 시행 후 신용대출, 주택담보대출, 전월세보증금 대출에서 유입액이 유출액을 크게 웃돌고 있으며, 신규 상품 출시와 경쟁심화 기조로 인해 성장세가 이어질 것으로 예상된다.' metadata={'company_name': '카카오뱅크', 'dir_route': 'datas/카카오뱅크_한국신용평가(2024.09.13)', 'file_name': '카카오뱅크_한국신용평가(2024.09.13).pdf', 'id': 51, 'investment': '한국신용평가(2024.09.13)', 'original_content': '대 출 갈아타기 시행 이후(신용대출은 2023년 5월 시행, 주택담보대출 및 전월세보증금 대출은 2024 년부터 시행), 신용대출, 주택담보대출, 전월세보증금 대출 각각 유입금액이 유출 금액을 크게 상회하 고 있다. 지속적으로 신규 상품을 출시하고 있는 점, 은행의 경쟁심화 기조가 유리하게 작용하고 있는 점 등을 고려할 때, 동행의 성장추세는 당분간 지속될 것으로 판단한다.', 'page': 3, 'type': 'paragraph'}
page_content='카카오뱅크는 높은 브랜드 인지도와 편리한 모바일 앱을 바탕으로 개인고객을 많이 확보했으며, 특히 주택담보대출 상품 출시 후 고객 기반이 더욱 넓어졌다. 그 결과 대출채권 규모가 빠른 속도로 증가해 2024년 6월 말에는 42.7조 원에 달했고, 이는 전년 대비 37.8%, 올해 상반기 대비 10.6% 상승한 수치이다. 이러한 성장세는 주로 주택담보대출 분야에서의 성과 덕분이며, 전체 대출채권 증가액 중 7.9조 원이 주택담보대출 증가분이었다.' metadata={'company_name': '카카오뱅크', 'dir_route': 'datas/카카오뱅크_한국신용평가(2024.09.13)', 'file_name': '카

In [2]:
eval_df = pipe.QA_eval(mode = 'ensemble', sampling = False) # sampling이 False일 경우, 모든 validation 데이터셋에 대해 평가를 진행합니다.
# 구현이 귀찮아서 일단 ./datas/g_eval_result_{model_name}.csv 경로에 저장되도록 하였습니다..

NameError: name 'pipe' is not defined

In [3]:
import pandas as pd
# retrieval G eval score과 generation G eval score의 점수 부분만 파싱합니다. (저장된 csv에는 전부 문자로 돼 있습니다.)

eval_df['generation_score'] = eval_df['generation_score'].apply(lambda x: x[x.rfind('/')-3:x.rfind('/')])
eval_df['retrieval_score'] = eval_df['retrieval_score'].apply(lambda x: x[x.rfind('/')-3:x.rfind('/')])
eval_df['generation_score'] = pd.to_numeric(eval_df['generation_score'], errors = 'coerce')
eval_df['retrieval_score'] = pd.to_numeric(eval_df['retrieval_score'], errors = 'coerce')

In [4]:
eval_df['generation_score'].describe()

count    264.000000
mean      18.329545
std        3.392191
min       10.000000
25%       16.000000
50%       18.000000
75%       19.000000
max       26.000000
Name: generation_score, dtype: float64